# Data Preparation

## COVID19 Case Dataset

Run the cell below to upload the dataset files using colab upload widget. Click the "Choose Files" button to select a local file to upload

In [1]:
from google.colab import files
uploaded = files.upload()

Read the csv file of "COVID19 Cases" dataset into a pandas dataframe and display the first 5 rows

In [2]:
import pandas as pd
import numpy as np
df_case = pd.read_csv('COVID19 cases.csv')
df_case.head()

,_id,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Currently Hospitalized,Currently in ICU,Currently Intubated,Ever Hospitalized,Ever in ICU,Ever Intubated
0,315771,1,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-22,2020-01-23,FEMALE,RESOLVED,No,No,No,No,No,No
1,315772,2,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-21,2020-01-23,MALE,RESOLVED,No,No,No,Yes,No,No
2,315773,3,Sporadic,20 to 29 Years,Parkwoods-Donalda,M3A,Travel,CONFIRMED,2020-02-05,2020-02-21,FEMALE,RESOLVED,No,No,No,No,No,No
3,315774,4,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-16,2020-02-25,FEMALE,RESOLVED,No,No,No,No,No,No
4,315775,5,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-20,2020-02-26,MALE,RESOLVED,No,No,No,No,No,No


Drop the columns that won't be used i.e. _id and FSA

In [3]:
df_case.drop(['_id','FSA'], axis=1, inplace=True)
df_case.shape

(29645, 16)

Replace NaN with 0. Drop the rows where there is no neighbourhood name present if any

In [4]:
df_case.fillna(0,inplace=True)
df_case.drop(df_case[df_case['Neighbourhood Name']==0].index, inplace=True)

Remove leading and trailing spaces in the neighbourhood names. Replace the spaces in neighbourhood names with "_"

In [5]:
df_case['Neighbourhood Name'] = df_case['Neighbourhood Name'].str.strip()
df_case['Neighbourhood Name'] = df_case['Neighbourhood Name'].str.replace(' ','_')

Replace the "Yes" or "No" string values in the 6 columns below with True or False boolean values

In [6]:
boolean_list = ['Currently in ICU','Currently Hospitalized','Currently Intubated','Ever in ICU','Ever Hospitalized','Ever Intubated']
for b in boolean_list:
  df_case[b] = df_case[b].apply(lambda x : True if x == 'Yes' else False)

Display the post-processing COVID19 Case dataset

In [7]:
df_case

,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Currently Hospitalized,Currently in ICU,Currently Intubated,Ever Hospitalized,Ever in ICU,Ever Intubated
0,1,Sporadic,50 to 59 Years,Willowdale_East,Travel,CONFIRMED,2020-01-22,2020-01-23,FEMALE,RESOLVED,False,False,False,False,False,False
1,2,Sporadic,50 to 59 Years,Willowdale_East,Travel,CONFIRMED,2020-01-21,2020-01-23,MALE,RESOLVED,False,False,False,True,False,False
2,3,Sporadic,20 to 29 Years,Parkwoods-Donalda,Travel,CONFIRMED,2020-02-05,2020-02-21,FEMALE,RESOLVED,False,False,False,False,False,False
3,4,Sporadic,60 to 69 Years,Church-Yonge_Corridor,Travel,CONFIRMED,2020-02-16,2020-02-25,FEMALE,RESOLVED,False,False,False,False,False,False
4,5,Sporadic,60 to 69 Years,Church-Yonge_Corridor,Travel,CONFIRMED,2020-02-20,2020-02-26,MALE,RESOLVED,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29640,30273,Sporadic,80 to 89 Years,Hillcrest_Village,Institutional,CONFIRMED,2020-10-21,2020-11-01,FEMALE,ACTIVE,True,False,False,True,False,False
29641,30274,Outbreak Associated,60 to 69 Years,Cabbagetown-South_St._James_Town,N/A - Outbreak associated,CONFIRMED,2020-10-22,2020-10-22,MALE,ACTIVE,False,False,False,False,False,False
29642,30275,Sporadic,50 to 59 Years,Glenfield-Jane_Heights,Unknown/Missing,CONFIRMED,2020-10-25,2020-10-25,MALE,ACTIVE,False,False,False,False,False,False
29643,30276,Sporadic,30 to 39 Years,Woburn,Close contact,CONFIRMED,2020-10-28,2020-10-28,MALE,ACTIVE,False,False,False,False,False,False


## Neighbourhood Dataset

Run the cell below to upload the Neighbourhood dataset. Click "Choose Files" button to upload a local file

In [8]:
from google.colab import files
uploaded = files.upload()

Read the neighbourhood csv file into a pandas dataframe and display the first 5 rows

In [9]:
df_nh = pd.read_csv('neighbourhood-profiles-2016-csv.csv')
df_nh.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,...,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,...,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,...,NIA,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,NIA,NIA,NIA,No Designation,No Designation,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,NIA,NIA,No Designation,No Designation,NIA,No Designation,NIA,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695","15,873","25,797","21,396","13,154","23,236","6,577","29,960","22,291","21,737","7,727","14,257","9,266","11,499","17,757","11,669","9,955","10,968","13,362","31,340","26,984","16,472","15,935","14,133","9,666","17,180","27,051","25,003","36,625","35,052",...,"10,803","10,529","22,246","14,974","20,923","46,496","10,070","9,941","16,724","21,849","27,876","17,812","24,623","25,051","27,446","15,683","21,567","10,360","21,108","16,556","7,607","17,510","65,913","27,392","33,312","26,274","17,992","11,098","27,917","50,434","16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177","26,918","15,434","19,348","17,671","13,530","23,185","6,488","27,876","21,856","22,057","7,763","14,302","8,713","11,563","17,787","12,053","9,851","10,487","13,093","28,349","24,770","14,612","15,703","13,743","9,444","16,712","26,739","24,363","34,631","34,659",...,"10,007","10,488","22,267","15,050","20,631","45,912"

Filter to only keep the rows for the desired topics/characteristics


Transpose the entire dataframe so that the topics/charateristics become columns and the neighbourhoods become rows

In [10]:
topic_list = ['Highest certificate, diploma or degree','Ethnic origin population','Main mode of commuting','Age characteristics']
characteristic_list = ['Population, 2016','Employment rate','Unemployment rate','  Average after-tax income of households in 2015 ($)','  2 persons','  3 persons','  4 persons','  5 or more persons']
df_nh.drop(['_id','Category','Data Source','City of Toronto'], axis=1, inplace=True)
df_nh = df_nh.loc[(df_nh['Topic'].isin(topic_list)) | df_nh['Characteristic'].isin(characteristic_list)]
df_nh = df_nh.T


Slightly modify the characteristic names (i.e. remove spaces or replace spaces with "_") based on their topic category, to be in sync with the individual names defined in our ontology

In [11]:
for i,col in enumerate(df_nh.columns):
  df_nh.iloc[1,i] = df_nh.iloc[1,i].strip()
  df_nh.iloc[1,i] = df_nh.iloc[1,i].replace("'",'')
  if df_nh.iloc[0,i] == 'Ethnic origin population':
    df_nh.iloc[1,i] = df_nh.iloc[1,i].replace(' ','')
  elif df_nh.iloc[0,i] in (['Highest certificate, diploma or degree','Main mode of commuting','Age characteristics','Household and dwelling characteristics']):
    df_nh.iloc[1,i] = df_nh.iloc[1,i].replace(' ','_')

Set the topics row to be the primary column header, and the characteristics row to be the secondary column header


Remove duplicated columns

Remove all the "," in value cells and convert their data type from string to numeric

Reset index

In [12]:
df_nh.columns = [df_nh.iloc[0].values, df_nh.iloc[1].values]
df_nh = df_nh.loc[:,~df_nh.columns.duplicated()]
df_nh.iloc[2:,:] = df_nh.iloc[2:,:].replace(',','',regex=True)
df_nh.iloc[2:,:] = df_nh.iloc[2:,:].apply(pd.to_numeric)
df_nh = df_nh.iloc[2:].reset_index(drop=False)

Remove leading and trailing spaces in the neighbourhood names. Replace the spaces in neighbourhood names with "_"

In [13]:
df_nh.loc[:,('index','')] = df_nh.loc[:,('index','')].str.strip()
df_nh.loc[:,('index','')] = df_nh.loc[:,('index','')].str.replace(' ','_')

Aggregate the COVID19 case dataset by neighbourhood and count each neighbourhood's cases 

In [14]:
case_count = df_case.groupby('Neighbourhood Name')['Assigned_ID'].agg('count')
df_case_count = pd.DataFrame({('Case Count',''):case_count})
df_case_count.reset_index(inplace=True)
df_case_count

,Neighbourhood Name,Case Count
,,
0,Agincourt_North,207
1,Agincourt_South-Malvern_West,132
2,Alderwood,97
3,Annex,249
4,Banbury-Don_Mills,112
...,...,...
135,Wychwood,132
136,Yonge-Eglinton,42
137,Yonge-St.Clair,53


Merge the above case number dataframe into the neighbourhood dataframe using neighbourhood name as key

Fill the place where case number is NaN with 0. It means the neighbourhood name does not appear in the COVID19 case dataset, and thus the neighourhood has 0 case reported

In [15]:
df_nh = pd.merge(df_case_count, df_nh, left_on='Neighbourhood Name',right_on='index',how='right')
df_nh['Neighbourhood Name'] = df_nh['index']
df_nh.drop([('index','')], axis=1, inplace=True)
# df_nh = df_nh.loc[:,~df_nh.columns.duplicated()]
df_nh.fillna(0, inplace=True)
df_nh.drop(df_nh[df_nh['Neighbourhood Name'] == 0].index, inplace=True)

Insert a new column called "Case to population ratio". Calculate its value by dividing case number by total population within each neigbourhood

In [16]:
df_nh.insert(2,'Case to Population Ratio','')
df_nh['Case to Population Ratio'] = np.array(df_nh[('Case Count','')])/np.array(df_nh[('Population and dwellings','Population, 2016')])


Display the post-processing neighbourhood dataframe

In [17]:
pd.options.display.max_columns=None
pd.options.display.max_rows=20
df_nh

Neighbourhood Name Case Count Case to Population Ratio  \
                                                                        
0                 Agincourt_North      207.0                 0.007110   
1    Agincourt_South-Malvern_West      132.0                 0.005556   
2                       Alderwood       97.0                 0.008047   
3                           Annex      249.0                 0.008157   
4               Banbury-Don_Mills      112.0                 0.004044   
..                            ...        ...                      ...   
135                      Wychwood      132.0                 0.009199   
136                Yonge-Eglinton       42.0                 0.003554   
137                Yonge-St.Clair       53.0                 0.004231   
138       York_University_Heights      717.0                 0.025985   
139            Yorkdale-Glen_Park      346.0                 0.023372   

    Population and dwellings   Age characteristics                      \
            Population, 2016 Children_(0-14_years) Youth_(15-24_years)   
0                    29113.0                3840.0              3705.0   
1                    23757.0                3075.0              3360.0   
2                    12054.0                1760.0              1235.0   
3                    30526.0                2360.0              3750.0   
4                    27695.0                3605.0              2730.0   
..                       ...                   ...                 ...   
135                  14349.0                1860.0              1320.0   
136                  11817.0                1800.0              1225.0   
137                  12528.0                1210.0               920.0   
138                  27593.0                4045.0              4750.0   
139                  14804.0                1960.0              1870.0   

                                                            \
    Working_Age_(25-54_years) Pre-retirement_(55-64_years)   
0                     11305.0                       4230.0   
1                      9965.0                       3265.0   
2                      5220.0                       1825.0   
3                     15040.0                       3480.0   
4                     10810.0                       3555.0   
..                        ...                          ...   
135                    6420.0                       1595.0   
136                    5860.0                       1325.0   
137                    5960.0                       1540.0   
138                   12290.0                       2965.0   
139                    5860.0                       1810.0   

                                                                       \
    Seniors_(65+_years) Older_Seniors_(85+_years) Male:_0_to_04_years   
0                6045.0                     925.0               660.0   
1                4105.0                     555.0               575.0   
2                2015.0                     320.0               360.0   
3                5910.0                    1040.0               445.0   
4                6975.0                    1640.0               570.0   
..                  ...                       ...                 ...   
135              3150.0                     880.0               325.0   
136              1600.0                     165.0               300.0   
137              2905.0                     470.0               220.0   
138              3530.0                     400.0               755.0   
139              3295.0                     775.0               320.0   

                                                                    \
    Male:_05_to_09_years Male:_10_to_14_years Male:_15_to_19_years   
0                  695.0                660.0                840.0   
1                  540.0                460.0                780.0   
2                  270.0                225.0                285.

# Generate Turtle File

Install the rdflib module using pip

In [18]:
!pip install rdflib

Import modules. Define namespaces

In [19]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, OWL, RDFS, RDF, XSD

xml = Namespace("http://www.w3.org/XML/1998/namespace")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
temp = Namespace("https://www.w3.org/TR/2020/CR-owl-time-20200326/")
cov = Namespace("http://ontology.eil.utoronto.ca/MIE1501/2020Project/Team6#")
time = Namespace("http://www.w3.org/2006/time#")


Iterate through the neighbourhood dataframe and add each row into a graph following the Subject, Property, Value pattern. Each row represents one neighbourhood. The value in the 'Neighbourhood Name' column is used as a unique identifier.

In [20]:
g=Graph()
for index, row in df_nh.iterrows():
    g.add((URIRef(cov+row[('Neighbourhood Name','')]), RDF.type, cov.Neighbourhood))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.hasName, Literal(row[('Neighbourhood Name','')], datatype=XSD.string)))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.averageAfterTaxHouseholdIncome, Literal(row[('Income of households in 2015','Average after-tax income of households in 2015 ($)')], datatype=XSD.double)))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.COVID19CaseToPopulationRatio, Literal(row[('Case to Population Ratio','')], datatype=XSD.double)))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.hasTotalPopulation, Literal(row[('Population and dwellings','Population, 2016')], datatype=XSD.double)))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]+'HouseholdCharacteristics'), RDF.type, cov.HouseholdCharacteristics))
    for i, new_df in df_nh['Household and dwelling characteristics'].iteritems():
        g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), RDF.type, cov.HouseholdCharacteristics))
        g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.hasCharacteristics, URIRef(cov+i)))
        g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.population, Literal(row[('Household and dwelling characteristics',i)], datatype=XSD.postiveInteger)))
        g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.hasHouseholdCharacteristics, URIRef(cov+row[('Neighbourhood Name','')]+i)))
    for i, new_df in df_nh['Age characteristics'].iteritems():
        g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), RDF.type, cov.AgeCharacteristics))
        g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.hasCharacteristics, URIRef(cov+i)))
        g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.population, Literal(row[('Age characteristics',i)], datatype=XSD.postiveInteger)))
        g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.hasAgeCharacteristics, URIRef(cov+row[('Neighbourhood Name','')]+i)))
    for i, new_df in df_nh['Highest certificate, diploma or degree'].iteritems():
        if i not in ['Total_-_Highest_certificate,_diploma_or_degree_for_the_population_aged_15_years_and_over_in_private_households_-_25%_sample_data','Total_-_Highest_certificate,_diploma_or_degree_for_the_population_aged_25_to_64_years_in_private_households_-_25%_sample_data']:
            g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), RDF.type, cov.HighestDegree))      
            g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.hasCharacteristics, URIRef(cov+i)))
            g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.population, Literal(row[('Highest certificate, diploma or degree',i)], datatype=XSD.positiveInteger)))
            g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.hasEducation, URIRef(cov+row[('Neighbourhood Name','')]+i)))      
    for i, new_df in df_nh['Main mode of commuting'].iteritems():
        if i not in ['Total_-_Main_mode_of_commuting_for_the_employed_labour_force_aged_15_years_and_over_in_private_households_with_a_usual_place_of_work_or_no_fixed_workplace_address_-_25%_sample_data']:
            g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), RDF.type, cov.MainModeofCommute))     
            g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.hasCharacteristics, URIRef(cov+i)))
            g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.population, Literal(row[('Main mode of commuting',i)], datatype=XSD.positiveInteger)))
            g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.hasMainModeofCommute, URIRef(cov+row[('Neighbourhood Name','')]+i)))
    for i, new_df in df_nh['Ethnic origin population'].iteritems():
        if i not in ['Total-Ethnicoriginforthepopulationinprivatehouseholds-25%sampledata']:
            g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), RDF.type, cov.EthnicOrigin))    
            g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.hasCharacteristics, URIRef(cov+i)))
            g.add((URIRef(cov+row[('Neighbourhood Name','')]+i), cov.population, Literal(row[('Ethnic origin population',i)], datatype=XSD.positiveInteger)))
            g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.hasEthnicOrigin, URIRef(cov+row[('Neighbourhood Name','')]+i)))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]+'Employed'), RDF.type, cov.EmploymentStatus))      
    g.add((URIRef(cov+row[('Neighbourhood Name','')]+'Employed'), cov.hasCharacteristics, URIRef(cov+'Employed')))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]+'Employed'), cov.hasRate, Literal(row[('Labour force status','Employment rate')], datatype=XSD.double)))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.hasEmploymentStatus, URIRef(cov+row[('Neighbourhood Name','')]+'Employed')))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]+'Unemployed'), RDF.type, cov.EmploymentStatus))      
    g.add((URIRef(cov+row[('Neighbourhood Name','')]+'Unemployed'), cov.hasCharacteristics, URIRef(cov+'Unemployed')))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]+'Unemployed'), cov.hasRate, Literal(row[('Labour force status','Unemployment rate')], datatype=XSD.double)))
    g.add((URIRef(cov+row[('Neighbourhood Name','')]), cov.hasEmploymentStatus, URIRef(cov+row[('Neighbourhood Name','')]+'Unemployed')))

In [21]:
# print(g.serialize(format='turtle').decode('UTF-8'))

Iterate through the COVID19 case dataframe and add each row into a graph following the Subject, Property, Value pattern. Each row represents one case. The value in the 'Assigned_ID' column is used as a unique identifier.

In [22]:
for index, row in df_case.iterrows():
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), RDF.type, cov.COVID19Case))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.assignedID, Literal(row['Assigned_ID'], datatype=XSD.positiveInteger)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.ageGroup, Literal(row['Age Group'], datatype=RDFS.Literal)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.classification, Literal(row['Classification'], datatype=XSD.string)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.currentlyInICU, Literal(row['Currently in ICU'], datatype=XSD.boolean)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.currentlyHospitalized, Literal(row['Currently Hospitalized'], datatype=XSD.boolean)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.currentlyIntubated, Literal(row['Currently Intubated'], datatype=XSD.boolean)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.everInICU, Literal(row['Ever in ICU'], datatype=XSD.boolean)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.everHospitalized, Literal(row['Ever Hospitalized'], datatype=XSD.boolean)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.everIntubated, Literal(row['Ever Intubated'], datatype=XSD.boolean)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), foaf.gender, Literal(row['Client Gender'], datatype=RDFS.Literal)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.sourceOfInfection, Literal(row['Source of Infection'], datatype=RDFS.Literal)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.outcome, Literal(row['Outcome'], datatype=XSD.string)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.outbreakAssociated, Literal(row['Outbreak Associated'], datatype=XSD.string)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.inNeighbourhood, URIRef(cov+row['Neighbourhood Name'])))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Episode Date'])), RDF.type, time.Instant))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.episodeDate, URIRef(cov+'Covid19_Case'+str(row['Episode Date']))))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Episode Date'])), time.inXSDDate, Literal(row['Episode Date'], datatype=XSD.date)))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Reported Date'])), RDF.type, time.Instant))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Assigned_ID'])), cov.reportedDate, URIRef(cov+'Covid19_Case'+str(row['Reported Date']))))
    g.add((URIRef(cov+'Covid19_Case'+str(row['Reported Date'])), time.inXSDDate, Literal(row['Reported Date'], datatype=XSD.date)))

Output the graph into a turtle file

In [23]:
g.serialize('ProjectTeam6_part2.ttl',format='turtle')

Run the cell below to download the generated turtle file

In [24]:
from google.colab import files
files.download('ProjectTeam6_part2.ttl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>